# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# !pip install cartopy
# !pip install geoviews
# !pip install pyproj

In [3]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests
import json

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,53.73,72,97,6.31,AU,1703958313
1,1,peruibe,-24.3200,-46.9983,74.88,84,100,4.29,BR,1703958313
2,2,hendersonville,36.3048,-86.6200,39.81,85,100,4.00,US,1703958040
3,3,kerikeri,-35.2268,173.9474,71.91,92,81,7.61,NZ,1703958315
4,4,businga,3.3333,20.8833,77.79,47,61,1.79,CD,1703958316


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=79)\
                            &(city_data_df['Max Temp']>=62)\
                            &(city_data_df['Cloudiness']<=60)\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='PK')
                            &(city_data_df['Humidity']<50),:]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
print(ideal_df)

     City_ID            City      Lat       Lng  Max Temp  Humidity  \
14        14         federal -30.9547  -58.7833     75.69        46   
23        23      rio grande -32.0350  -52.0986     73.51        45   
74        74     el calafate -50.3408  -72.2768     63.93        45   
82        82        khorixas -20.3667   14.9667     75.58        25   
91        91       taoudenni  22.6783   -3.9836     71.35        18   
109      109    ciudad acuna  29.3232 -100.9522     63.41        40   
144      144          illizi  26.4833    8.4667     67.80        22   
170      170  puerto natales -51.7236  -72.4875     64.85        42   
232      232         magaria  12.9983    8.9099     73.85        19   
262      262           nasir  15.5894   32.5634     73.85        18   
277      277          bechar  31.6167   -2.2167     64.72        17   
279      279      montevideo -34.8335  -56.1674     68.02        45   
316      316       taroudant  30.4703   -8.8770     62.49        22   
317   

### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,federal,AR,-30.9547,-58.7833,46,
23,rio grande,BR,-32.0350,-52.0986,45,
74,el calafate,AR,-50.3408,-72.2768,45,
82,khorixas,NaN,-20.3667,14.9667,25,
91,taoudenni,ML,22.6783,-3.9836,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel', 'filter':'', 'bias':'', 'apiKey': geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
federal - nearest hotel: No hotel found
rio grande - nearest hotel: Hotel Vila Moura Executivo
el calafate - nearest hotel: Hostería Las Avutardas
khorixas - nearest hotel: iGowati Country Hotel and Lodge
taoudenni - nearest hotel: No hotel found
ciudad acuna - nearest hotel: Hotel San Jorge
illizi - nearest hotel: بوناقة
puerto natales - nearest hotel: Dorotea Patagonia Hostel
magaria - nearest hotel: No hotel found
nasir - nearest hotel: منزل طارق
bechar - nearest hotel: Hôtel Madania
montevideo - nearest hotel: Hotel Escuela
taroudant - nearest hotel: Hotel Tiouit
adrar - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
chui - nearest hotel: Alerces
villa juarez - nearest hotel: No hotel found
biltine - nearest hotel: فندق ثلاث نجوم بلتن
gao - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
bardai - nearest hotel: No hotel found
timbuktu - nearest hotel: Hotel La Maison
opuwo - nearest hotel: Ohakane Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
14,federal,AR,-30.9547,-58.7833,46,No hotel found
23,rio grande,BR,-32.0350,-52.0986,45,Hotel Vila Moura Executivo
74,el calafate,AR,-50.3408,-72.2768,45,Hostería Las Avutardas
82,khorixas,NaN,-20.3667,14.9667,25,iGowati Country Hotel and Lodge
91,taoudenni,ML,22.6783,-3.9836,18,No hotel found
109,ciudad acuna,MX,29.3232,-100.9522,40,Hotel San Jorge
144,illizi,DZ,26.4833,8.4667,22,بوناقة
170,puerto natales,CL,-51.7236,-72.4875,42,Dorotea Patagonia Hostel
232,magaria,NE,12.9983,8.9099,19,No hotel found
262,nasir,SD,15.5894,32.5634,18,منزل طارق


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity')

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)